<a href="https://colab.research.google.com/github/pedromperezc/CEIA/blob/master/NLP/Desafio_2_Bot_DNN_Spacy_esp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Bot con NLTK utilizando un corpus de wikipedia


In [ ]:
#instalo librería stanza para procesar documentos en castellano
!pip install stanza

     |████████████████████████████████| 432 kB 5.5 MB/s 
     |████████████████████████████████| 175 kB 36.2 MB/s 
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171046 sha256=4d10e2ba0b1a96cf70b193f541c6363e183497145a4ce86d91c425f97a80b41e
  Stored in directory: /root/.cache/pip/wheels/8a/4e/b6/57b01db010d17ef6ea9b40300af725ef3e210cb1acfb7ac8b6
Successfully built emoji


In [ ]:
# Installo spacy-stanza
!pip install spacy_stanza

     |████████████████████████████████| 6.0 MB 5.5 MB/s 
     |████████████████████████████████| 451 kB 39.9 MB/s 
     |████████████████████████████████| 181 kB 45.2 MB/s 
     |████████████████████████████████| 10.1 MB 8.2 MB/s 
     |████████████████████████████████| 653 kB 37.9 MB/s 
     |████████████████████████████████| 42 kB 407 kB/s 
  Attempting uninstall: catalogue
    Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Attempting uninstall: srsly
    Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [ ]:
!pip install unidecode

     |████████████████████████████████| 235 kB 5.3 MB/s 


In [ ]:
import json
import string
import random
import re
import urllib.request
import unidecode

import numpy as np
import tensorflow as tf 
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout

# Para leer y parsear el texto en HTML de wikipedia
import bs4 as bs
import nltk

import stanza
import spacy_stanza
import spacy

nltk.download('punkt')
nltk.download('wordnet')

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
# Descargo diccionarios para stanza en español:
stanza.download("es")

#LInicializo el lematizador de stanza
nlp = spacy_stanza.load_pipeline("es")

2022-03-23 20:27:43 INFO: Downloading default packages for language: es (Spanish)...
2022-03-23 20:27:44 INFO: File exists: /root/stanza_resources/es/default.zip.
2022-03-23 20:27:51 INFO: Finished downloading models and saved to /root/stanza_resources.
2022-03-23 20:27:51 INFO: Loading these models for language: es (Spanish):
| Processor | Package |
-----------------------
| tokenize  | ancora  |
| mwt       | ancora  |
| pos       | ancora  |
| lemma     | ancora  |
| depparse  | ancora  |
| ner       | conll02 |

2022-03-23 20:27:51 INFO: Use device: gpu
2022-03-23 20:27:51 INFO: Loading: tokenize
2022-03-23 20:27:51 INFO: Loading: mwt
2022-03-23 20:27:51 INFO: Loading: pos
2022-03-23 20:27:51 INFO: Loading: lemma
2022-03-23 20:27:51 INFO: Loading: depparse
2022-03-23 20:27:52 INFO: Loading: ner
2022-03-23 20:27:53 INFO: Done loading processors!


### Datos
Se consumira los datos del artículo de wikipedia sobre el covid en español

In [ ]:
raw_html = urllib.request.urlopen('https://es.wikipedia.org/wiki/COVID-19')
raw_html = raw_html.read()

article_html = bs.BeautifulSoup(raw_html, 'lxml')

article_paragraphs = article_html.find_all('p')

article_text = ''

for para in article_paragraphs:
    article_text += para.text

article_text = article_text.lower()

In [ ]:
# Demos un vistazo
article_text

'la enfermedad por coronavirus de 2019,[3]\u200b más conocida como covid-19, covid-19[nota 1]\u200b o covid, e incorrectamente llamada neumonía por coronavirus, coronavirus[nota 2]\u200b o corona, es una enfermedad infecciosa causada por el sars-cov-2.[11]\u200b[12]\u200b \nproduce síntomas que incluyen fiebre, tos,[13]\u200b disnea (dificultad respiratoria), mialgia (dolor muscular)[14]\u200by fatiga.[15]\u200b[16]\u200b en casos graves se caracteriza por producir neumonía, síndrome de dificultad respiratoria aguda,[17]\u200b sepsis[18]\u200b y choque circulatorio. choque séptico es la forma más común en estos casos, pero los otros tipos también pueden ocurrir. por ejemplo, choque obstructivo puede resultar de embolia pulmonar, una complicación de covid-19.[19]\u200b según la oms, la infección es mortal entre el 0,5\xa0% y el 1\xa0% de los casos.[20]\u200b no existe tratamiento específico; las medidas terapéuticas principales consisten en aliviar los síntomas y mantener las funciones 

In [ ]:
print("Cantidad de caracteres en la nota:", len(article_text))

Cantidad de caracteres en la nota: 73768


### 2 - Preprocesamiento
- Remover caracteres especiales
- Quitar espacios o saltos

In [ ]:
text = re.sub(r'\[[0-9]*\]', ' ', article_text)
text = re.sub(r'\s+', ' ', text)
text = re.sub(r'\u200b', '', text)
text = re.sub(r'\[nota [0-9]]', '', text)

In [ ]:
# Demos un vistazo
text

'la enfermedad por coronavirus de 2019,  más conocida como covid-19, covid-19 o covid, e incorrectamente llamada neumonía por coronavirus, coronavirus o corona, es una enfermedad infecciosa causada por el sars-cov-2.   produce síntomas que incluyen fiebre, tos,  disnea (dificultad respiratoria), mialgia (dolor muscular) y fatiga.   en casos graves se caracteriza por producir neumonía, síndrome de dificultad respiratoria aguda,  sepsis  y choque circulatorio. choque séptico es la forma más común en estos casos, pero los otros tipos también pueden ocurrir. por ejemplo, choque obstructivo puede resultar de embolia pulmonar, una complicación de covid-19.  según la oms, la infección es mortal entre el 0,5 % y el 1 % de los casos.  no existe tratamiento específico; las medidas terapéuticas principales consisten en aliviar los síntomas y mantener las funciones vitales.  la transmisión del sars-cov-2 se produce mediante pequeñas gotas —microgotas de flügge — que se emiten al hablar, estornudar

In [ ]:
print("Cantidad de caracteres en el texto:", len(text))

Cantidad de caracteres en el texto: 72137


### 3 - Dividir el texto en sentencias y en palabras

In [ ]:
corpus = nltk.sent_tokenize(text)
words = nltk.word_tokenize(text)

In [ ]:
# Demos un vistazo
corpus[:10]

['la enfermedad por coronavirus de 2019,  más conocida como covid-19, covid-19 o covid, e incorrectamente llamada neumonía por coronavirus, coronavirus o corona, es una enfermedad infecciosa causada por el sars-cov-2.',
 'produce síntomas que incluyen fiebre, tos,  disnea (dificultad respiratoria), mialgia (dolor muscular) y fatiga.',
 'en casos graves se caracteriza por producir neumonía, síndrome de dificultad respiratoria aguda,  sepsis  y choque circulatorio.',
 'choque séptico es la forma más común en estos casos, pero los otros tipos también pueden ocurrir.',
 'por ejemplo, choque obstructivo puede resultar de embolia pulmonar, una complicación de covid-19.',
 'según la oms, la infección es mortal entre el 0,5 % y el 1 % de los casos.',
 'no existe tratamiento específico; las medidas terapéuticas principales consisten en aliviar los síntomas y mantener las funciones vitales.',
 'la transmisión del sars-cov-2 se produce mediante pequeñas gotas —microgotas de flügge — que se emiten

In [ ]:
# Demos un vistazo
words[:20]

['la',
 'enfermedad',
 'por',
 'coronavirus',
 'de',
 '2019',
 ',',
 'más',
 'conocida',
 'como',
 'covid-19',
 ',',
 'covid-19',
 'o',
 'covid',
 ',',
 'e',
 'incorrectamente',
 'llamada',
 'neumonía']

In [ ]:
print("Vocabulario:", len(words))

Vocabulario: 12824


### 4 - Funciones de ayuda para limpiar y procesar el input del usuario
- Lematizar los tokens de la oración
- Quitar símbolos de puntuación

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def perform_lemmatization(tokens):
    return [lemmatizer.lemmatize(token) for token in tokens]


def get_processed_text(document):
    punctuation_removal = dict((ord(punctuation), None) for punctuation in string.punctuation)

    # 1 - reduce el texto a mínuscula
    document_lower = document.lower()
    # 2 - quitar los simbolos de puntuacion
    document_puntuation = document_lower.translate(punctuation_removal)

    # 3 - Remuevo acentos
    word = unidecode.unidecode(document_puntuation) 

    # 3 - realiza la tokenización con stanza
    tokens = nlp (word)
    # 4 - realiza la lematización con stanza
    return [token.lemma_ for token in tokens]

In [ ]:
document = corpus[3]
document

'choque séptico es la forma más común en estos casos, pero los otros tipos también pueden ocurrir.'

In [ ]:
# se modifico la función get_processed_text para lematizar utilizando la librería stanza ya que sklearn no lo posee.
get_processed_text(document)

['choque',
 'septico',
 'ser',
 'el',
 'forma',
 'mas',
 'comun',
 'en',
 'este',
 'caso',
 'pero',
 'el',
 'otro',
 'tipo',
 'tambien',
 'poder',
 'ocurrir']

In [ ]:
# Utilizo el diccionario de stop words de spacy
spacy_stopwords = spacy.lang.es.stop_words.STOP_WORDS
stop_words = []
for word in spacy_stopwords:
  stop_words.append(unidecode.unidecode(word))

print('Numero stop words: %d' % len(stop_words))
print('Primeras 10 stop words: %s' % list(stop_words)[:10])

Numero stop words: 551
Primeras 10 stop words: ['tienen', 'podran', 'consigues', 'este', 'empleas', 'claro', 'pasada', 'excepto', 'eran', 'dejo']


### 5 - Utilizar vectores TF-IDF y la similitud coseno construido con el corpus de wikipedia

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def generate_response(user_input, corpus):
    response = ''
    # Sumar al corpus la pregunta del usuario para calcular
    # su cercania con otros documentos/sentencias
    corpus.append(user_input)

    # Crear un vectorizar TFIDF que quite las "stop words" de español
    # nuestra funcion para obtener los tokens lematizados "get_processed_text"
    word_vectorizer = TfidfVectorizer(tokenizer=get_processed_text, stop_words=stop_words)
    
    # Crear los vectores a partir del corpus
    all_word_vectors = word_vectorizer.fit_transform(corpus)

    # Calcular la similitud coseno entre todas los documentos excepto el agregado (el útlimo "-1")
    # NOTA: con los word embedings veremos más en detalle esta matriz de similitud
    similar_vector_values = cosine_similarity(all_word_vectors[-1], all_word_vectors)

    # Obtener el índice del vector más cercano a nuestra oración
    # --> descartando la similitud contra nuestor vector propio
    similar_sentence_number = similar_vector_values.argsort()[0][-2]
    matched_vector = similar_vector_values.flatten()
    matched_vector.sort()
    vector_matched = matched_vector[-2]

    if vector_matched == 0:
        response = "I am sorry, I could not understand you"
    else:
        response = corpus[similar_sentence_number]
    
    corpus.remove(user_input)
    return response

### 6 - Ensayar el sistema
El sistema intentará encontrar la parte del artículo que más se relaciona con nuestro texto de entrada. Sugerencias ensayar:
- Tratamiento
- sintomas
- mortalidad
- Propagación

In [ ]:
# Se utilizará gradio para ensayar el bot
# Herramienta poderosa para crear interfaces rápidas para ensayar modelos
# https://gradio.app/
import sys
!{sys.executable} -m pip install gradio --quiet

     |████████████████████████████████| 668 kB 4.8 MB/s 
     |████████████████████████████████| 255 kB 46.5 MB/s 
     |████████████████████████████████| 53 kB 1.8 MB/s 
     |████████████████████████████████| 84 kB 2.3 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 1.1 MB 32.0 MB/s 
     |████████████████████████████████| 211 kB 44.2 MB/s 
     |████████████████████████████████| 2.0 MB 35.6 MB/s 
     |████████████████████████████████| 144 kB 49.2 MB/s 
     |████████████████████████████████| 271 kB 50.6 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 58 kB 5.1 MB/s 
     |████████████████████████████████| 79 kB 7.1 MB/s 
     |████████████████████████████████| 43 kB 1.9 MB/s 
     |████████████████████████████████| 61 kB 405 kB/s 
     |████████████████████████████████| 3.6 MB 34.1 MB/s 
     |████████████████████████████████| 856 kB 42.7 MB/s 
     |█████████████████████████

In [ ]:
import gradio as gr

def bot_response(human_text):
    print(human_text)
    return generate_response(human_text.lower(), corpus)

iface = gr.Interface(
    fn=bot_response,
    inputs=["textbox"],
    outputs="text",
    layout="vertical")

iface.launch(debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://20730.gradio.app

This share link expires in 72 hours. For free permanent hosting, check out Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.


(<fastapi.applications.FastAPI at 0x7f29e85cd6d0>,
 'http://127.0.0.1:7860/',
 'https://20730.gradio.app')

In [ ]:
human_text = "tratamientos"
print(generate_response(human_text.lower(), corpus))

varios tratamientos experimentales contra el covid-19 están actualmente en desarrollo o han sido aprobados en al menos un país: otros tratamientos que se consideraron prometedores durante los primeros meses de la pandemia, como la hidroxicloroquina y el lopinavir/ritonavir, resultaron después ser inefectivos o incluso perjudiciales.


In [ ]:
human_text = "sintomas"
print(generate_response(human_text.lower(), corpus))

los síntomas de la covid-19 son inespecíficos y su presentación, según la oms, puede incluso carecer de síntomas (asintomático).


In [ ]:
human_text = "mortalidad"
print(generate_response(human_text.lower(), corpus))

los eventos tromboembólicos con mortalidad asociada son un riesgo para las embarazadas.


In [ ]:
human_text = "propagacion"
print(generate_response(human_text.lower(), corpus))

se recomienda lavarse las manos para evitar la propagación del coronavirus.


In [ ]:
human_text = "transmision"
print(generate_response(human_text.lower(), corpus))

actualmente, son muchos los estudios que se centran en evaluar la posible transmisión vertical.


In [ ]:
human_text = "quien fue el Paciente cero"
print(generate_response(human_text.lower(), corpus))


el imperial college de londres calculó un r0=1,5 a 3,5.  se llama caso índice o "paciente cero" al caso que da lugar a la atención del investigador y origina acciones para conocer un foco de infección; corresponde al primero caso confirmado por la autoridad sanitaria y conduce (indica) hacia un brote localizado.


In [ ]:
human_text = "que pruebas de laboratorio se pueden realizar para diagnosticarl el covid"
print(generate_response(human_text.lower(), corpus))

se obtuvieron muestras de estos pacientes para realizar análisis de laboratorio.


In [ ]:
human_text = "cuales son las medidas preventivas"
print(generate_response(human_text.lower(), corpus))

algunas organizaciones internacionales, como la oms, han publicado medidas preventivas para reducir la transmisión del virus.


In [ ]:
human_text = "que es el distanciamiento social"
print(generate_response(human_text.lower(), corpus))

el distanciamiento social y el aislamiento de las poblaciones infectadas pueden contener la epidemia.


# Conclusiones:
-	Se realizó la implementación para calcular la distancia coseno de un documento y compararlo con todos los documentos de un corpus utilizando el idioma castellano.
-	A través de las librerías stanza se logró implementar un lematizador de tokens en castellano.
-	Se utilizó la librería spacy para obtener un listado de stop words.
-	Se tuvo que aplicar un paso adicional en el pre procesamiento de los documentos para eliminar los acentos en palabras.
-	Se observó que las frases ingresadas tienen bastante relación con el documento propuesto como más cercano.


